In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Conv1D, LSTM, BatchNormalization, Dropout, GRU
import random

In [2]:
data = pd.read_csv('acc-data.csv')
data

Seq No.    Location   AccX   AccY    AccZ  Classs
0           1  Kidnapping  1.073 -1.992  10.420       1
1           2  Kidnapping -0.459 -3.524   9.807       1
2           3  Kidnapping -0.153 -2.911   8.581       1
3           4  Kidnapping -1.072 -3.524   9.347       1
4           5  Kidnapping -1.072 -2.758   8.887       1
...       ...         ...    ...    ...     ...     ...
1945       46      Stairs -2.604  3.064  12.105       0
1946       47      Stairs -4.749  3.064  13.484       0
1947       48      Stairs -1.991  2.298   6.129       0
1948       49      Stairs -1.838  1.685   6.895       0
1949       50      Stairs -1.838  3.524   8.734       0

[1950 rows x 6 columns]

In [3]:
data.columns

Index(['Seq No.', 'Location', 'AccX', 'AccY', 'AccZ', 'Classs'], dtype='object')

In [5]:
len(data[data['Classs']==1])/50

10.0

In [6]:
n = 39
X = np.empty((n,50*3))
Y = np.empty(n)
for i in range(0,n):
    for j in range(0,3):
        X[i,j*50:(j+1)*50]=data.iloc[i*50:(i+1)*50,j+2]
    Y[i]=data.iloc[i*50,5]
    
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train, Y_test = train_test_split(X,Y, test_size=0.3,random_state=8, shuffle=True)
X_train.shape

(27, 150)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [8]:
lr = LogisticRegression()
lr.fit(X_train,Y_train)
pred_y = lr.predict(X_test)
print(accuracy_score(Y_test,pred_y))
print(precision_score(Y_test,pred_y))
print(recall_score(Y_test,pred_y))

0.5833333333333334
0.0
0.0


In [9]:
from sklearn.tree import DecisionTreeClassifier
lr = DecisionTreeClassifier()
lr.fit(X_train,Y_train)
pred_y = lr.predict(X_test)
print(accuracy_score(Y_test,pred_y))
print(precision_score(Y_test,pred_y))
print(recall_score(Y_test,pred_y))

0.75
0.3333333333333333
0.5


In [10]:
from sklearn.ensemble import RandomForestClassifier
lr = RandomForestClassifier()
lr.fit(X_train,Y_train)
pred_y = lr.predict(X_test)
print(accuracy_score(Y_test,pred_y))
print(precision_score(Y_test,pred_y))
print(recall_score(Y_test,pred_y))

0.6666666666666666
0.0
0.0


In [11]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0)
clf.fit(X_train,Y_train)
pred_y = lr.predict(X_test)
print(accuracy_score(Y_test,pred_y))
print(precision_score(Y_test,pred_y))
print(recall_score(Y_test,pred_y))

0.6666666666666666
0.0
0.0


In [12]:
n = 39
X = np.empty((n,50,3))
Y = np.empty(n)
for i in range(0,n):
    X[i]=data.iloc[i*50:(i+1)*50,2:5]
    Y[i]=data.iloc[i*50,5]
    
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train, Y_test = train_test_split(X,Y, test_size=0.3,random_state=8, shuffle=True)
X_train.shape

(27, 50, 3)

In [13]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.random import set_seed

def get_model():
    set_seed(2)
    model = Sequential([
        Dense(50, activation='relu',input_shape=(50,3)),        
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(loss='binary_crossentropy',
                optimizer=Adam(learning_rate=0.01),metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
    return model

In [14]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [15]:
X_train.shape

(27, 50, 3)

In [16]:
k=3
import numpy as np
size = X_train.shape[0] // k
avg_scores = []
for i in range(k):
    
    print("fold : ", i)
    model = get_model()
    
    X_train_ = np.concatenate([X_train[:i*size] , X_train[(i+1)*size:]],axis=0)
    X_val = X_train[i*size: (i+1)*size]
    
    Y_train_ = np.concatenate([Y_train[:i*size] , Y_train[(i+1)*size:]])
    Y_val = Y_train[i*size: (i+1)*size]
    
    model = get_model()
    
    model.fit(X_train_,Y_train_,epochs=12)
    scores = model.evaluate(X_val, Y_val)
    avg_scores.append(scores)

np.average(avg_scores,axis=0)

fold :  0
Epoch 1/12
1/1 [==============================] - 2s 2s/step - loss: 1.2994 - accuracy: 0.3889 - precision_1: 0.2857 - recall_1: 0.8000
Epoch 2/12
1/1 [==============================] - 0s 7ms/step - loss: 2.9671 - accuracy: 0.7222 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
Epoch 3/12
1/1 [==============================] - 0s 7ms/step - loss: 1.0784 - accuracy: 0.7222 - precision_1: 0.5000 - recall_1: 0.2000
Epoch 4/12
1/1 [==============================] - 0s 5ms/step - loss: 1.0178 - accuracy: 0.8889 - precision_1: 0.7143 - recall_1: 1.0000
Epoch 5/12
1/1 [==============================] - 0s 5ms/step - loss: 1.2954 - accuracy: 0.8889 - precision_1: 0.7143 - recall_1: 1.0000
Epoch 6/12
1/1 [==============================] - 0s 7ms/step - loss: 1.0345 - accuracy: 0.8889 - precision_1: 0.7143 - recall_1: 1.0000
Epoch 7/12
1/1 [==============================] - 0s 5ms/step - loss: 0.6541 - accuracy: 0.9444 - precision_1: 0.8333 - recall_1: 1.0000
Epoch 8/12
1/1 [========

array([3.88763261, 0.77777779, 0.66666667, 0.27777778])

In [17]:
model = get_model()
model.fit(X_train,Y_train,epochs=12)

Epoch 1/12
1/1 [==============================] - 1s 1s/step - loss: 1.1202 - accuracy: 0.5185 - precision_6: 0.3684 - recall_6: 0.8750
Epoch 2/12
1/1 [==============================] - 0s 7ms/step - loss: 4.8196 - accuracy: 0.7037 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 3/12
1/1 [==============================] - 0s 6ms/step - loss: 2.7724 - accuracy: 0.7037 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 4/12
1/1 [==============================] - 0s 8ms/step - loss: 0.8377 - accuracy: 0.8148 - precision_6: 0.6667 - recall_6: 0.7500
Epoch 5/12
1/1 [==============================] - 0s 6ms/step - loss: 1.4236 - accuracy: 0.8148 - precision_6: 0.6154 - recall_6: 1.0000
Epoch 6/12
1/1 [==============================] - 0s 5ms/step - loss: 1.4054 - accuracy: 0.8148 - precision_6: 0.6154 - recall_6: 1.0000
Epoch 7/12
1/1 [==============================] - 0s 5ms/step - loss: 0.7836 - accuracy: 0.8889 - precision_6: 0.7273 - recall_6: 1.0000
Epoch 8/12
1/1 [==========

In [18]:
score=model.evaluate(X_test,Y_test)
print(model.metrics_names[1])
score[1]

1/1 [==============================] - 1s 811ms/step - loss: 2.0456 - accuracy: 0.7500 - precision_6: 0.3333 - recall_6: 0.5000
accuracy


0.75